## Create Business Analyst Agent

In this notebook we will first create a business analyst agent with Amazon Bedrock Agents that will be able to detect AI/MLuse cases in a given database.

Then we will evaluate the agents's performance.


In [ ]:
# Import necessary libraries and load environment variables
from dotenv import load_dotenv, find_dotenv, set_key
import os
import sagemaker
import boto3
import pandas as pd


# loading environment variables that are stored in local file
local_env_filename = 'dev.env'
load_dotenv(find_dotenv(local_env_filename),override=True)

os.environ['REGION'] = os.getenv('REGION')
os.environ['S3_BUCKET_NAME'] = os.getenv('S3_BUCKET_NAME')
os.environ['AWS_ACCOUNT'] = os.getenv('AWS_ACCOUNT')
os.environ['BUSINESSANALYST_AGENT_PROFILE_ARN'] = os.getenv('BUSINESSANALYST_AGENT_PROFILE_ARN')
os.environ['BUSINESSANALYST_AGENT_EVAL_PROFILE_ARN'] = os.getenv('BUSINESSANALYST_AGENT_EVAL_PROFILE_ARN')

REGION = os.environ['REGION']
S3_BUCKET_NAME = os.environ['S3_BUCKET_NAME']
AWS_ACCOUNT = os.environ['AWS_ACCOUNT']
BUSINESSANALYST_AGENT_PROFILE_ARN = os.environ['BUSINESSANALYST_AGENT_PROFILE_ARN']
BUSINESSANALYST_AGENT_EVAL_PROFILE_ARN = os.environ['BUSINESSANALYST_AGENT_EVAL_PROFILE_ARN']

MODEL_ID =  "anthropic.claude-3-5-sonnet-20240620-v1:0"

In [3]:
import botocore.config
config = botocore.config.Config(
    connect_timeout=600,  # 10 minutes
    read_timeout=600,     # 10 minutes
    retries={'max_attempts': 3}
)

session = boto3.Session(region_name=REGION)

# Create a SageMaker session
sagemaker_session = sagemaker.Session(boto_session=session)
bedrock_agent_client = session.client('bedrock-agent', config=config)
bedrock_agent_runtime_client = session.client('bedrock-agent-runtime', config=config)
bedrock_runtime_client = session.client('bedrock-runtime', config=config)
bedrock_client = session.client('bedrock', config=config)
lambda_client = session.client('lambda', config=config)
iam_resource = session.resource('iam')
iam_client = session.client('iam')
athena_client = session.client('athena')
s3_client = session.client('s3')


## Create Business Analyst agent lambda function

In [ ]:
%%writefile ../businessanalyst/bedrock_business_analyst_agent.py
import json
import sys
import logging
import pandas as pd
import boto3
import os
import tempfile
from typing import Optional, List, Dict, Any, Tuple
from pydantic import BaseModel
import time

class Parameters(BaseModel):
    AthenaDatabase: Optional[str] = None

# Configure logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# get the environment variables
if 'S3_BUCKET_NAME' not in globals():
    S3_BUCKET_NAME = os.getenv('S3_BUCKET_NAME')
    logger.info(f"S3_BUCKET_NAME: {S3_BUCKET_NAME}")

if 'MODEL_ID' not in globals():
    MODEL_ID = os.getenv('MODEL_ID')
    logger.info(f"MODEL_ID: {MODEL_ID}")

if 'ATHENA_QUERY_EXECUTION_LOCATION' not in globals():
    ATHENA_QUERY_EXECUTION_LOCATION = f's3://{S3_BUCKET_NAME}/athena_results/'
    logger.info(f"ATHENA_QUERY_EXECUTION_LOCATION: {ATHENA_QUERY_EXECUTION_LOCATION}")

# check if session python variable exists
if 'SESSION_PROFILE' in globals():
    logger.info(f"Session profile found: {SESSION_PROFILE}")
    session = boto3.Session(profile_name=SESSION_PROFILE, region_name=REGION)
else:
    logger.info('No session profile found, using default session')
    session = boto3.Session()

s3_client = session.client('s3')
athena_client = session.client('athena')



logger.info('start athena result location configuration')
try:
    response = athena_client.get_work_group(WorkGroup='primary')
    ConfigurationUpdates={}
    ConfigurationUpdates['EnforceWorkGroupConfiguration']= True
    ResultConfigurationUpdates= {}
    athena_location = "s3://"+ S3_BUCKET_NAME +"/athena_results/"
    ResultConfigurationUpdates['OutputLocation']=athena_location
    EngineVersion = response['WorkGroup']['Configuration']['EngineVersion']
    ConfigurationUpdates['ResultConfigurationUpdates']=ResultConfigurationUpdates
    ConfigurationUpdates['PublishCloudWatchMetricsEnabled']= response['WorkGroup']['Configuration']['PublishCloudWatchMetricsEnabled']
    ConfigurationUpdates['EngineVersion']=EngineVersion
    ConfigurationUpdates['RequesterPaysEnabled']= response['WorkGroup']['Configuration']['RequesterPaysEnabled']
    response2 = athena_client.update_work_group(WorkGroup='primary',ConfigurationUpdates=ConfigurationUpdates,State='ENABLED')
    logger.info(f"athena output location updated to s3://{S3_BUCKET_NAME}/athena_results/")  
except Exception as e:
    logger.error(str(e))

class BusinessAnalystTools:
    """Collection of tools for the Business Analyst Agent to use"""
    
    def __init__(self, session, s3_bucket_name: str, athena_database: str):
        logger.info(f"Initializing BusinessAnalystTools with bucket: {s3_bucket_name}, database: {athena_database}")
        self.s3_client = session.client('s3')
        self.athena_client = session.client('athena')
        self.s3_bucket_name = s3_bucket_name
        self.athena_database = athena_database
        
    def get_database_schema(self) -> str:
        """Retrieve the SQL database schema from S3"""
        schema_prefix = 'metadata/sql_table_definition'
        logger.info(f"Retrieving database schema from s3://{self.s3_bucket_name}/{schema_prefix}")
        
        sql_database_schema = []
        try:
            response = self.s3_client.list_objects_v2(
                Bucket=self.s3_bucket_name, 
                Prefix=schema_prefix
            )
            
            if 'Contents' not in response:
                logger.warning(f"No schema files found in s3://{self.s3_bucket_name}/{schema_prefix}")
                return "[]"
            
            logger.info(f"Found {len(response['Contents'])} schema files")
            
            for item in response['Contents']:
                if item['Key'].endswith('/'):
                    continue
                    
                logger.info(f"Reading schema file: {item['Key']}")
                try:
                    content = self.s3_client.get_object(
                        Bucket=self.s3_bucket_name, 
                        Key=item['Key']
                    )['Body'].read().decode('utf-8')
                    sql_database_schema.append(content)
                    logger.debug(f"Successfully read schema from {item['Key']}")
                except Exception as e:
                    logger.error(f"Error reading schema file {item['Key']}: {str(e)}")
            
            logger.info(f"Successfully retrieved {len(sql_database_schema)} schema definitions")
            return json.dumps(sql_database_schema)
            
        except Exception as e:
            logger.error(f"Error in get_database_schema: {str(e)}", exc_info=True)
            return "[]"

    def get_use_cases(self) -> List[Dict]:
        """Retrieve available AI/ML use cases from S3"""
        use_cases_path = 'metadata/use_cases/use_case_details.jsonl'
        logger.info(f"Retrieving use cases from s3://{self.s3_bucket_name}/{use_cases_path}")
        
        try:
            with tempfile.NamedTemporaryFile() as tmp:
                self.s3_client.download_file(
                    self.s3_bucket_name,
                    use_cases_path,
                    tmp.name
                )
                df = pd.read_json(tmp.name, lines=True)
                use_cases = df.to_dict('records')
                logger.info(f"Successfully retrieved {len(use_cases)} use cases")
                logger.debug(f"Use cases: {json.dumps(use_cases, indent=2)}")
                return use_cases
        except Exception as e:
            logger.error(f"Error in get_use_cases: {str(e)}", exc_info=True)
            return []

    def execute_query(self, query: str) -> Tuple[str, Optional[pd.DataFrame], Optional[str]]:
        """Execute an Athena query and return results with detailed error info"""
        logger.info(f"Executing Athena query in database {self.athena_database}")
        logger.debug(f"Query: {query}")
        
        try:
            response = self.athena_client.start_query_execution(
                QueryString=query,
                QueryExecutionContext={
                    'Database': self.athena_database,
                    'Catalog': 'AwsDataCatalog'
                },
                ResultConfiguration={
                    'OutputLocation': f's3://{self.s3_bucket_name}/athena_results/'
                }
            )
            
            query_id = response['QueryExecutionId']
            logger.info(f"Query execution started with ID: {query_id}")
            
            # Wait for completion
            while True:
                status = self.athena_client.get_query_execution(QueryExecutionId=query_id)
                state = status['QueryExecution']['Status']['State']
                logger.debug(f"Query state: {state}")
                
                if state in ['SUCCEEDED', 'FAILED', 'CANCELLED']:
                    break
                time.sleep(1)
            
            # Get detailed error information if query failed
            error_info = None
            if state != 'SUCCEEDED':
                status_details = status['QueryExecution']['Status']
                error_info = {
                    'state': state,
                    'reason': status_details.get('StateChangeReason', 'Unknown error'),
                    'athena_error': status_details.get('AthenaError', {}),
                    'query_id': query_id
                }
                error_msg = (
                    f"Query failed with state {state}.\n"
                    f"Reason: {error_info['reason']}\n"
                    f"Query ID: {query_id}"
                )
                if 'AthenaError' in status_details:
                    error_msg += f"\nAthena Error: {status_details['AthenaError']}"
                logger.error(error_msg)
                return state, None, error_msg
            
            # Query succeeded
            logger.info(f"Query {query_id} completed successfully")
            results = self.athena_client.get_query_results(QueryExecutionId=query_id)
            df = self._convert_results_to_df(results)
            logger.info(f"Query returned {len(df)} rows and {len(df.columns)} columns")
            return 'SUCCEEDED', df, None
                
        except Exception as e:
            error_msg = f"Error executing query: {str(e)}"
            logger.error(error_msg, exc_info=True)
            return 'ERROR', None, error_msg

    def save_dataset(self, df: pd.DataFrame, use_case_name: str) -> str:
        """Save a dataset to S3 and return its location"""
        logger.info(f"Saving dataset for use case: {use_case_name}")
        logger.debug(f"Dataset shape: {df.shape}")
        
        try:
            with tempfile.NamedTemporaryFile(suffix='.csv') as tmp:
                df.to_csv(tmp.name, index=False)
                s3_path = f'ml_datasets/{use_case_name}_dataset.csv'
                
                logger.info(f"Uploading dataset to s3://{self.s3_bucket_name}/{s3_path}")
                self.s3_client.upload_file(
                    tmp.name,
                    self.s3_bucket_name,
                    s3_path
                )
                
                location = f's3://{self.s3_bucket_name}/{s3_path}'
                logger.info(f"Dataset successfully saved to {location}")
                return location
                
        except Exception as e:
            logger.error(f"Error in save_dataset: {str(e)}", exc_info=True)
            return ''

    def _convert_results_to_df(self, query_results: Dict) -> pd.DataFrame:
        """Convert Athena query results to a pandas DataFrame"""
        try:
            columns = [col['Name'] for col in query_results['ResultSet']['ResultSetMetadata']['ColumnInfo']]
            logger.debug(f"Converting query results with columns: {columns}")
            
            data = []
            for row in query_results['ResultSet']['Rows'][1:]:  # Skip header row
                data.append([item.get('VarCharValue', '') for item in row['Data']])
            
            df = pd.DataFrame(data, columns=columns)
            logger.debug(f"Converted results to DataFrame with shape: {df.shape}")
            return df
            
        except Exception as e:
            logger.error(f"Error in _convert_results_to_df: {str(e)}", exc_info=True)
            return pd.DataFrame()

    def execute_and_save_query(self, query: str, use_case_name: str = None) -> Tuple[str, Optional[Dict]]:
        """Execute query, save full results, and return samples"""
        logger.info(f"Executing and saving query for {use_case_name if use_case_name else 'analysis'}")
        logger.debug(f"Query: {query}")
        
        try:
            status, df, error_msg = self.execute_query(query)
            
            if status == 'SUCCEEDED' and df is not None:
                result_info = {
                    "status": status,
                    "total_rows": len(df),
                    "total_columns": len(df.columns),
                    "columns": list(df.columns),
                    "sample_data": df.head(5).to_dict('records')  # Only return 5 sample rows
                }
                
                # Save the full dataset if we have a use case name
                if use_case_name and not df.empty:
                    dataset_location = self.save_dataset(df, use_case_name)
                    result_info["dataset_location"] = dataset_location
                    logger.info(f"Saved full dataset ({len(df)} rows) to {dataset_location}")
                
                return status, result_info
            else:
                return status, {
                    "status": status,
                    "error": error_msg
                }
                
        except Exception as e:
            error_msg = f"Error in execute_and_save_query: {str(e)}"
            logger.error(error_msg, exc_info=True)
            return 'ERROR', {
                "status": 'ERROR',
                "error": error_msg
            }

def truncate_response(data: Any, max_size: int = 20000) -> Any:
    """Truncate response data to stay within size limits"""
    if isinstance(data, dict):
        serialized = json.dumps(data)
        if len(serialized) <= max_size:
            return data
            
        # For dictionary responses, try to preserve structure while reducing content
        truncated = data.copy()
        if 'data' in truncated and isinstance(truncated['data'], list):
            # Calculate approximate size per record
            record_count = len(truncated['data'])
            if record_count > 0:
                avg_record_size = len(json.dumps(truncated['data'])) / record_count
                # Calculate how many records we can keep
                safe_record_count = int((max_size * 0.8) / avg_record_size)  # 80% of max size
                truncated['data'] = truncated['data'][:safe_record_count]
                truncated['truncated'] = True
                truncated['total_records'] = record_count
                truncated['showing_records'] = safe_record_count
                return truncated
                
    elif isinstance(data, list):
        serialized = json.dumps(data)
        if len(serialized) <= max_size:
            return data
            
        # For list responses, truncate the list
        original_length = len(data)
        # Calculate approximate size per item
        if original_length > 0:
            avg_item_size = len(serialized) / original_length
            safe_item_count = int((max_size * 0.8) / avg_item_size)  # 80% of max size
            return {
                'data': data[:safe_item_count],
                'truncated': True,
                'total_items': original_length,
                'showing_items': safe_item_count
            }
    
    return data

def lambda_handler(event: Dict[str, Any], context: Any) -> Dict[str, Any]:
    try:
        logger.info(f"Received event: {json.dumps(event)}")
        
        # Extract parameters from requestBody if present
        parameters_dict = {}
        if 'requestBody' in event and 'content' in event['requestBody']:
            content = event['requestBody']['content']
            if 'application/json' in content and 'properties' in content['application/json']:
                for prop in content['application/json']['properties']:
                    parameters_dict[prop['name']] = prop['value']
        
        # Create Parameters object with the extracted values
        parameters = Parameters(**parameters_dict)

        # Initialize tools
        tools = BusinessAnalystTools(
            session=session,
            s3_bucket_name=S3_BUCKET_NAME,
            athena_database=parameters.AthenaDatabase
        )
        
        # Extract APIPath from event
        api_path = event.get('apiPath', '').strip('/')
        
        logger.info(f"API Path: {api_path}")
        logger.info(f"Parameters: {parameters}")

        response_data = None

        if api_path == 'GetDatabaseSchema':
            # Get database schema
            schema = tools.get_database_schema()
            response_data = json.loads(schema)  # Convert string to JSON array
            
        elif api_path == 'GetUseCases':
            # Get available use cases
            use_cases = tools.get_use_cases()
            response_data = use_cases
            
        elif api_path == 'ExecuteQuery':
            # Execute Athena query
            query = parameters_dict.get('Query')
            use_case_name = parameters_dict.get('UseCaseName')  # Optional parameter
            
            if not query:
                raise ValueError("Query parameter is required")
                
            status, result_info = tools.execute_and_save_query(query, use_case_name)
            if status != 'SUCCEEDED':
                # Instead of raising ValueError, return the error info directly
                logger.info(f"Query failed with info: {result_info}")
                response_data = result_info
            else:
                response_data = result_info
            
        elif api_path == 'SaveDataset':
            # This endpoint becomes optional since datasets are saved automatically
            # but keep it for explicit saves
            use_case_name = parameters_dict.get('UseCaseName')
            data = parameters_dict.get('Data')
            
            if not use_case_name or not data:
                raise ValueError("UseCaseName and Data parameters are required")
                
            df = pd.DataFrame(data)
            location = tools.save_dataset(df, use_case_name)
            response_data = {"location": location}
            
        else:
            raise ValueError(f"Unknown API path: {api_path}")
                
        # Check and truncate response size if needed
        response_size = sys.getsizeof(json.dumps(response_data))
        if response_size > 20000:  # 20KB limit
            logger.warning(f"Response size {response_size} exceeds limit. Truncating content...")
            response_data = truncate_response(response_data)
                
        response_body = {
            'application/json': {
                'body': response_data
            }
        }
        
        # Set response code based on status if it exists
        response_code = 200
        if isinstance(response_data, dict) and response_data.get('status') in ['FAILED', 'CANCELLED', 'ERROR']:
            response_code = 400

        action_response = {
            'actionGroup': event['actionGroup'],
            'apiPath': event['apiPath'],
            'httpMethod': event['httpMethod'],
            'httpStatusCode': response_code,
            'responseBody': response_body
        }

        return {'messageVersion': '1.0', 'response': action_response}
            
    except ValueError as e:
        # Handle bad request errors (400)
        logger.error(f"Validation error: {str(e)}")
        return {
            "messageVersion": "1.0",
            "response": {
                "actionGroup": event.get('actionGroup'),
                "apiPath": event.get('apiPath'),
                "httpMethod": event.get('httpMethod', 'POST'),
                "httpStatusCode": 400,
                "responseBody": {
                    "application/json": {
                        "body": {
                            "error": str(e)
                        }
                    }
                }
            }
        }
    except Exception as e:
        # Handle internal server errors (500)
        logger.error(f"Internal error: {str(e)}", exc_info=True)
        return {
            "messageVersion": "1.0",
            "response": {
                "actionGroup": event.get('actionGroup'),
                "apiPath": event.get('apiPath'),
                "httpMethod": event.get('httpMethod', 'POST'),
                "httpStatusCode": 500,
                "responseBody": {
                    "application/json": {
                        "body": {
                            "error": f"Internal server error: {str(e)}"
                        }
                    }
                }
            }
        }


In [ ]:
%%writefile ../businessanalyst/businessanalyst.Dockerfile
FROM public.ecr.aws/lambda/python:3.11

# Install build dependencies first
RUN yum install libgomp git gcc gcc-c++ make -y \
 && yum clean all -y && rm -rf /var/cache/yum


RUN python3 -m pip --no-cache-dir install --upgrade --trusted-host pypi.org --trusted-host files.pythonhosted.org pip \
 && python3 -m pip --no-cache-dir install --upgrade wheel setuptools \
 && python3 -m pip --no-cache-dir install --upgrade pandas \
 && python3 -m pip --no-cache-dir install --upgrade boto3 \
 && python3 -m pip --no-cache-dir install --upgrade opensearch-py \
 && python3 -m pip --no-cache-dir install --upgrade Pillow \
 && python3 -m pip --no-cache-dir install --upgrade pyarrow \
 && python3 -m pip --no-cache-dir install --upgrade fastparquet \
 && python3 -m pip --no-cache-dir install --upgrade urllib3 \
 && python3 -m pip --no-cache-dir install --upgrade pydantic

# Copy function code
WORKDIR /var/task
COPY ../businessanalyst/bedrock_business_analyst_agent.py .
COPY ../notebooks/utils/ utils/ 

# Set handler environment variable
ENV _HANDLER="bedrock_business_analyst_agent.lambda_handler"

# Let's go back to using the default entrypoint
ENTRYPOINT [ "/lambda-entrypoint.sh" ]
CMD [ "bedrock_business_analyst_agent.lambda_handler" ]

## Build and run local docker container to test the businessanalyst-lambda function

In [ ]:
# Build and run local docker container
!docker build -t businessanalyst-lambda -f ../businessanalyst/businessanalyst.Dockerfile ..

In [ ]:
# docker run with tailing log
credentials = session.get_credentials()
credentials = credentials.get_frozen_credentials()

!docker run -d \
-e AWS_ACCESS_KEY_ID={credentials.access_key} \
-e AWS_SECRET_ACCESS_KEY={credentials.secret_key} \
-e AWS_SESSION_TOKEN={credentials.token} \
-e AWS_DEFAULT_REGION={REGION} \
-e REGION={REGION} \
-e AWS_LAMBDA_FUNCTION_TIMEOUT=900 \
-e S3_BUCKET_NAME={S3_BUCKET_NAME} \
-e MODEL_ID={MODEL_ID} \
-p 9000:8080 businessanalyst-lambda

In [ ]:
!docker ps --filter ancestor=businessanalyst-lambda

In [ ]:
# detect SQL database schema
request_body = {
    "apiPath": "/GetDatabaseSchema",
    "requestBody": {
        "content": {
            "application/json": {
                "properties": [
                    {
                        "name": "AthenaDatabase",
                        "type": "string",
                        "value": f"{S3_BUCKET_NAME.replace('-', '_')}"
                    }
                ]
            }
        }
    },
    "httpMethod": "POST",
    "actionGroup": "BusinessAnalystActions",
}

import requests
response = requests.post("http://localhost:9000/2015-03-31/functions/function/invocations",
                         json=request_body,
                         timeout=900  # 15 minutes timeout
)
print(response.json())


In [ ]:
# get use case catalog
request_body = {
    "apiPath": "/GetUseCases",
    "requestBody": {
        "content": {
            "application/json": {
                "properties": [
                    {
                        "name": "AthenaDatabase",
                        "type": "string",
                        "value": f"{S3_BUCKET_NAME.replace('-', '_')}"
                    }
                ]
            }
        }
    },
    "httpMethod": "POST",
    "actionGroup": "BusinessAnalystActions",
}

import requests
response = requests.post("http://localhost:9000/2015-03-31/functions/function/invocations",
                         json=request_body,
                         timeout=900  # 15 minutes timeout
)
print(response.json())


In [ ]:
# stop the container
!docker stop $(docker ps -q --filter ancestor=businessanalyst-lambda)
!docker ps --filter ancestor=businessanalyst-lambda


## Upload docker image to ECR

In [12]:
## Create ECR repository for dataengineer-lambda (if not already created in 1_environmentSetup.ipynb)
#!aws ecr create-repository --repository-name automatedinsights/lambda_businessanalyst --region {REGION} --profile {SESSION_PROFILE}

In [ ]:
# Upload docker image to ECR
!aws ecr get-login-password --region {REGION} --profile {SESSION_PROFILE} | docker login --username AWS --password-stdin {AWS_ACCOUNT}.dkr.ecr.{REGION}.amazonaws.com
!docker tag businessanalyst-lambda:latest {AWS_ACCOUNT}.dkr.ecr.{REGION}.amazonaws.com/automatedinsights/lambda_businessanalyst:latest
!docker push {AWS_ACCOUNT}.dkr.ecr.{REGION}.amazonaws.com/automatedinsights/lambda_businessanalyst:latest

## Create & Test Bedrock Agent

In [ ]:
import logging
import random
import string
import json
from utils.bedrock_agent import BedrockAgentScenarioWrapper

logging.basicConfig(level=logging.INFO, format="%(levelname)s: %(message)s")
agent_name = "BusinessAnalyst"
MODEL_ID = "us.anthropic.claude-3-5-sonnet-20240620-v1:0"
prompt = f"Review the data and identify AI/ML use cases that can be performed on the data in the Athena database name: {S3_BUCKET_NAME.replace('-', '_')}"


instruction = """# Optimized AI Agent Instructions

## Role and Purpose
You are an expert business analyst specializing in AI/ML solution development. Your task is to identify high-value machine learning opportunities from available data sources.

## Core Capabilities
- Analyze SQL database schemas
- Execute and refine SQL queries against Athena databases
- Evaluate potential ML use cases for feasibility with available data
- Focus specifically on classification and regression problem types

## Primary Objective
Identify and thoroughly document ONE high-value machine learning use case that can be implemented with the available data.

## Analysis Workflow
1. Review the database schema to understand available tables and relationships
2. Explore data characteristics through sample queries
3. Identify a promising classification or regression use case
4. Develop and test an Athena SQL query that produces the required dataset
5. Document the complete use case specification

## Required Deliverable Components
For your identified ML use case, provide:

1. **Use Case Name**: Clear, descriptive title
2. **Description**: Concise explanation of the ML problem and approach
3. **Business Justification**: Specific business value and expected outcomes
4. **Target Column Specification**: Precise definition of what you're predicting
5. **ML Dataset Location**: Exact storage location of the prepared dataset
6. **Validated Athena SQL Query**: Working query that generates the complete dataset with target column

## Query Development Protocol
- Ensure all SQL follows Amazon Athena syntax standards
- Test your SQL query in Athena before finalizing
- If errors occur, analyze the error message and revise the query
- Maximum 3 attempts to fix any query issues
- If query cannot be resolved after 3 attempts, select an alternative use case

## Final Verification
Before submitting your final response, verify that:
- The SQL query executes successfully in Athena
- All required components are included in your documentation
- The ML dataset location is explicitly specified"""

postfix = "".join(
    random.choice(string.ascii_lowercase + "0123456789") for _ in range(8)
)

agent_name = agent_name + "_" + postfix

IMAGE_URI = f'{AWS_ACCOUNT}.dkr.ecr.{REGION}.amazonaws.com/automatedinsights/lambda_businessanalyst:latest'

agentCollaboration = 'DISABLED' #'SUPERVISOR' #|'SUPERVISOR_ROUTER'|'DISABLED'

sub_agents_list = []
promptOverrideConfiguration = None


lambda_environment_variables = {
    "S3_BUCKET_NAME": S3_BUCKET_NAME,
    "MODEL_ID": MODEL_ID
}

scenario = BedrockAgentScenarioWrapper(
    bedrock_agent_client=bedrock_agent_client,
    runtime_client=bedrock_agent_runtime_client,
    lambda_client=lambda_client,
    iam_resource=iam_resource,
    postfix=postfix,
    agent_name=agent_name,
    model_id=MODEL_ID,
    sub_agents_list=sub_agents_list,
    prompt=prompt,
    lambda_image_uri=IMAGE_URI,
    lambda_environment_variables=lambda_environment_variables,
    action_group_schema_path="action_groups/businessanalyst_open_api_schema.yml",
    instruction=instruction,
    agentCollaboration=agentCollaboration,
    promptOverrideConfiguration=promptOverrideConfiguration
)
try:
    scenario.run_scenario()
except Exception as e:
    logging.exception(f"Something went wrong: {e}")

In [15]:
# prompt = f"Review the data and identify AI/ML use cases that can be performed on the data in the Athena database name: {S3_BUCKET_NAME.replace('-', '_')}"

# scenario.prompt = prompt
# print(scenario.prompt)

# scenario.chat_with_agent()

## Agent Evaluation

In [ ]:
agent =scenario.agent

AGENT_ID = agent.get('agentId')

# get agent alias id
agent_aliases = bedrock_agent_client.list_agent_aliases(agentId= AGENT_ID)

AGENT_ALIAS_ID =  agent_aliases.get('agentAliasSummaries')[0].get('agentAliasId')
print(f"AGENT_ID: {AGENT_ID}")
print(f"AGENT_ALIAS_ID: {AGENT_ALIAS_ID}")

In [17]:
# save agent config to json file for evaluation
agent_config = {
    "agent_id": AGENT_ID,
    "agent_alias_id": AGENT_ALIAS_ID,
    "human_id": "User",
    "agent_name": "BusinessAnalyst",
    "agent_instruction": instruction,
    "tools": [
        {
            "tool_name": "BusinessAnalystAPI",
            "name": "BusinessAnalystAPI",
            "description": "Business Analyst API for database analysis and ML dataset preparation",
            "actions": [
                {
                    "name": "GetDatabaseSchema",
                    "description": "Retrieve the SQL database schema from S3",
                    "input_schema": {
                        "data_type": "object",
                        "properties": {
                            "AthenaDatabase": {
                                "data_type": "string",
                                "description": "The Athena database name",
                                "required": []
                            }
                        },
                        "required": [
                            "AthenaDatabase"
                        ]
                    },
                    "output_schema": {
                        "data_type": "array",
                        "items": {
                            "data_type": "object",
                            "properties": {
                                "table_name": {
                                    "data_type": "string"
                                },
                                "columns": {
                                    "data_type": "array",
                                    "items": {
                                        "data_type": "object",
                                        "properties": {
                                            "name": {
                                                "data_type": "string"
                                            },
                                            "type": {
                                                "data_type": "string"
                                            }
                                        }
                                    }
                                }
                            }
                        }
                    },
                    "requires_confirmation": False,
                    "meta": {}
                },
                {
                    "name": "GetUseCases",
                    "description": "Retrieve available AI/ML use cases from S3",
                    "output_schema": {
                        "data_type": "array",
                        "items": {
                            "data_type": "object",
                            "properties": {
                                "name": {
                                    "data_type": "string",
                                    "description": "Name of the use case"
                                },
                                "description": {
                                    "data_type": "string",
                                    "description": "Detailed description of the use case"
                                },
                                "required_columns": {
                                    "data_type": "array",
                                    "items": {
                                        "data_type": "string"
                                    },
                                    "description": "Required columns for this use case"
                                }
                            }
                        }
                    },
                    "requires_confirmation": False,
                    "meta": {}
                },
                {
                    "name": "ExecuteQuery",
                    "description": "Execute an Athena query, save results, and return samples",
                    "input_schema": {
                        "data_type": "object",
                        "properties": {
                            "AthenaDatabase": {
                                "data_type": "string",
                                "description": "The Athena database name",
                                "required": []
                            },
                            "Query": {
                                "data_type": "string",
                                "description": "The SQL query to execute",
                                "required": []
                            },
                            "UseCaseName": {
                                "data_type": "string",
                                "description": "Optional use case name to save full results",
                                "required": []
                            }
                        },
                        "required": [
                            "AthenaDatabase",
                            "Query"
                        ]
                    },
                    "output_schema": {
                        "data_type": "object",
                        "properties": {
                            "status": {
                                "data_type": "string",
                                "enum": ["SUCCEEDED", "FAILED", "CANCELLED", "ERROR"]
                            },
                            "total_rows": {
                                "data_type": "integer",
                                "description": "Total number of rows in the full result"
                            },
                            "total_columns": {
                                "data_type": "integer",
                                "description": "Total number of columns"
                            },
                            "columns": {
                                "data_type": "array",
                                "items": {
                                    "data_type": "string"
                                },
                                "description": "List of column names"
                            },
                            "sample_data": {
                                "data_type": "array",
                                "items": {
                                    "data_type": "object",
                                    "additionalProperties": True
                                },
                                "description": "Sample rows from the query results"
                            },
                            "dataset_location": {
                                "data_type": "string",
                                "description": "S3 location of saved dataset (if use case provided)"
                            },
                            "error": {
                                "data_type": "string",
                                "description": "Detailed error message if query failed"
                            }
                        }
                    },
                    "requires_confirmation": False,
                    "meta": {}
                },
                {
                    "name": "SaveDataset",
                    "description": "Save a dataset to S3",
                    "input_schema": {
                        "data_type": "object",
                        "properties": {
                            "UseCaseName": {
                                "data_type": "string",
                                "description": "Name of the use case for the dataset",
                                "required": []
                            },
                            "Data": {
                                "data_type": "array",
                                "items": {
                                    "data_type": "object",
                                    "additionalProperties": True
                                },
                                "description": "Dataset to save as array of records",
                                "required": []
                            }
                        },
                        "required": [
                            "UseCaseName",
                            "Data"
                        ]
                    },
                    "output_schema": {
                        "data_type": "object",
                        "properties": {
                            "location": {
                                "data_type": "string",
                                "description": "S3 location where the dataset was saved"
                            }
                        }
                    },
                    "requires_confirmation": False,
                    "meta": {}
                }
            ],
            "tool_type": "Module",
            "meta": {}
        }
    ],
    "reachable_agents": []
}
# save agent config to json file
with open('../businessanalyst/agent.json', 'w') as f:
    json.dump(agent_config, f, indent=4)


In [18]:
# define different evaluation scenarios
evaluation_scenarios = {
    "scenarios": [
        {
            "scenario": "DetectUseCases",
            "input_problem": f"Detect AI/ML use cases that can be performed on the data. Include the reasoning and the S3 location of the prepared dataset in the final response. Athena database name: {S3_BUCKET_NAME.replace('-', '_')}",   
            "assertions": [
                "agent: GetDatabaseSchema is executed to detect AI/ML use cases that can be performed on the data in the Athena database",
                "agent: GetUseCases is executed to detect AI/ML use cases that can be performed on the data in the Athena database",
                "agent: ExecuteQuery is executed to create a ML dataset",
                "agent: SaveDataset is executed to save the ML dataset to S3",
                "agent: The AI/ML use cases along with the respective S3 location of the ML dataset(s) and target column name are returned in the final response"
            ]
        }
        
    ]
}

# save evaluation scenarios to json file
with open('../businessanalyst/scenarios.json', 'w') as f:
    json.dump(evaluation_scenarios, f, indent=4)


In [ ]:
# Run the agent evaluation
from utils.benchmark import run_agent_evaluation

dataset_dir = "../businessanalyst"
results = run_agent_evaluation(
    scenario_filepath = f"{dataset_dir}/scenarios.json",
    agent_filepath = f"{dataset_dir}/agent.json",
    llm_judge_id = BUSINESSANALYST_AGENT_EVAL_PROFILE_ARN,
    region = REGION,
    session = session
)

# Check if results is not None before proceeding
if results is not None:
    # Create high-level metrics DataFrame
    metrics_df = pd.DataFrame({
        'user_gsr': [results['user_gsr']],
        'system_gsr': [results['system_gsr']],
        'overall_gsr': [results['overall_gsr']],
        'partial_gsr': [results['partial_gsr']],
        'scenario_count': [results['scenario_count']],
        'conversation_count': [results['conversation_count']]
    })

    # Create detailed assertions DataFrame
    assertions_list = []
    for eval_result in results['conversation_evals']:
        trajectory_index = eval_result['trajectory_index']
        for assertion in eval_result['report']:
            assertions_list.append({
                'trajectory_index': trajectory_index,
                'assertion_type': assertion['assertion_type'],
                'assertion': assertion['assertion'],
                'answer': assertion['answer'],
                'evidence': assertion['evidence']
            })

    assertions_df = pd.DataFrame(assertions_list)

    # Display results
    print("High-level Metrics:")
    display(metrics_df)

    print("\nDetailed Assertions:")
    display(assertions_df)

else:
    print("Error: Please check for errors in the evaluation.")

## Summary

- We first created a Docker container that contains all of the available tools/functions that the agent can use.
- We then created a Bedrock Agent with an Action Group that uses the Docker container in a Lambda function as the execution environment.
- We then created a set of evaluation scenarios that cover different aspects of the agent's behavior.
- We then ran the agent evaluation and reviewed the results.
